In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'isic-2024-challenge:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F63056%2F9094797%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240902%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240902T021436Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Daccb17b34cdb8266b58b2df3757027d44b954bd8c95863766cf12b39cc4d81531b3caf625841d671d9eaf2cf9ec12f3c7b6af86b5d3f186ad49082e8d7070ca1d6d3ac2b02bbac8a5d6754261c7beaecbb9d47ecfecba9a586a0cc557abb532058db18320125d17086ce5ba4f6ece886dc46808d1a7a0e6c6f91ef34de2adb065861eb277e23995dcb68fe67191641dab0cd9d9332c4c14c49f9cdc84195b85b3d95366692c9626c95f8c0eb2b5562440b3eb4c0d43a480414bdf23268161aa0291c357ef502da96e1e92a67a9760c4ca5d41513ec8ae772d6af4273d9ac010c87a0a485dcd82d2607567042584f1607269672245e2309a9b04aa08398779011,isic-2024-fails:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5406640%2F8982084%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240902%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240902T021436Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da898192b54804c115192fe8d0c3139c9563154b56f6bf6baa6d51f9df8a0f25b0bfe287e7f01d4efc5e5a8e6fb928023d3877e11f56fc11a5c60eb7d6b9ac00bb10c8d85ce7aa8a1a88d5147200dfdeda590019e0d52d9bde15c1d127fbeaa9ecd1c09bb3849b185d8299fbd6edc5a95cecb332b4326a9060a64e575c8dfa3874705e100eceed5386aed47ba8577a6db03df60614ce1bcda97f34f8cb40b8a7de959c41d77b0cd8edc7234488e45cd821d7779db597c22a1d1763ccc3003cd8d248ec43aa836b1a6841f89b2fe9fa883d220d8bf82607d7d09fee5efb44974d69b8c87f80ade0097502fe20f723e3b47ada20e41e8bece137cf9cc0a76418cc5,isic-2024-pl-submission-script-and-preds:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5415918%2F8991790%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240902%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240902T021436Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2a64443cf799a1ff3f19e61d5e6c87c9b889a1d9e7f04144f9b1665ab6ec99e8539de82c31533442113b7c7298ea121458fd0b93ae4be36914904f98ad5b03c27062bd6cf08bf8193e240be640fab14e59826156966a981e9f2591a1a5dea33699ff2d5af808755e4de8ed5d581dcd78aa34c6deb844d147c045a3357c64f52c0b0f9d12b96952e48d46763a9be753f546bdcbbea1c612255d0c68bbb14aab90aebf66d028bd51b2220a1e4c547cf0a5f2c07eb7e82d7caee839874ae191c35303f2a9f1ff9e851f7cb6069095a3cade2f930424fe76df1306dfb80d25283ef4379e1cff769d83275e1eb37f63b0ab442f9c431b842e35b3204ac7299c62bb81'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Initialization</h1></span>
This note book is forked from [here](https://www.kaggle.com/code/motono0223/isic-pytorch-training-baseline-image-only)

Credit:
* https://www.kaggle.com/code/samu2505/isic-pytorchmodel-training
    * This notebook added images from past ISIC datasets and I used it to enlarge the size of positive cases
    
Future Improvements:
* I used EffNet_b3, and maybe other model could have better performance.
* The size of dataset is limited due to the computation power. if you have better machine, you could enlarge the dataset of training.
* Parameter finetuning (the epoch = 20 now).
* I only one fold out of five, which can be changed.

Notebooks:
- Training notebook (This Notebook)
- [Inference notebook](https://www.kaggle.com/code/hugowjd/isic-2024-pytorch-inference-effnet-b2/)


## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 50px 50px;">Import Required Libraries</h1></span>

In [ ]:

!pip install torcheval
!pip install timm
!pip install torchmetrics


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import os
import gc
import cv2
import math
import copy
import time
import random
import glob
from pathlib import Path

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision
from torcheval.metrics.functional import binary_auroc
from torchmetrics import AUROC

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
# from colorama import Fore, Back, Style
# b_ = Fore.BLUE
# sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# Plot package
import PIL
from matplotlib import pyplot as plt
import seaborn as sns

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Configuration</h1></span>

In [ ]:
CONFIG = {
    "seed": 817,
    "epochs": 5,
    "img_size": 224,
    "model_name": "efficientnet_b4",
    "num_classes": 1,
    "train_batch_size": 64,
    "valid_batch_size": 32,
    "learning_rate": 1e-5,
    "scheduler": 'CosineAnnealingLR',
    "min_lr": 1e-9,
    "T_max": 500,
    "weight_decay": 1e-7,
    "fold" : 0,
    "n_fold": 100,
    "n_accumulate": 1,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "use_previous_positive_data": True,
}

In [ ]:
CONFIG['device']
timm.list_models()

# Table data

In [ ]:
# import pandas as pd
# import numpy as np
# df_train = pd.read_csv("/kaggle/input/isic-2024-challenge/train-metadata.csv")
# df_test = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')
# df_test['target'] = 0 # dummy
# print(df_train.describe())
# print(df_test.describe())
# # df_sub = pd.read_csv(SAMPLE)
# feature_name=['isic_id',
#         'age_approx',
#         'sex',
#         'anatom_site_general', 'clin_size_long_diam_mm',
#         'tbp_tile_type',
#         'tbp_lv_A', 'tbp_lv_Aext',
#         'tbp_lv_B', 'tbp_lv_Bext',
#         'tbp_lv_C', 'tbp_lv_Cext',
#         'tbp_lv_H', 'tbp_lv_Hext',
#         'tbp_lv_L', 'tbp_lv_Lext',
#         'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
#         'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
#         'tbp_lv_deltaL', 'tbp_lv_deltaLB',
#         'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity',
#         'tbp_lv_location', 'tbp_lv_location_simple',
#         'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence',
#         'tbp_lv_norm_border', 'tbp_lv_norm_color',
#         'tbp_lv_perimeterMM',
#         'tbp_lv_radial_color_std_max',
#         'tbp_lv_stdL', 'tbp_lv_stdLExt',
#         'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
#         'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z',
#         'target'
#         ]
# df_train=df_train[feature_name]
# df_test=df_test[feature_name]
# print(df_train.isnull().sum())
# print(df_test.isnull().sum())
# def data_cleaning(df):
#     df['age_approx'] = df['age_approx'].fillna(df['age_approx'].median())
#     df.drop(columns=['tbp_lv_location_simple'], inplace=True)
#     df['tbp_lv_location'] = df['tbp_lv_location'].replace('Unknown', np.nan)
#     return df

# # 清洗数据
# df_train_clean = data_cleaning(df_train)
# df_test_clean = data_cleaning(df_test)

# # 合并训练集和测试集进行编码
# combined_df = pd.concat([df_train_clean, df_test_clean], ignore_index=True)

# # 获取分类列的列表
# categorical_cols = ['anatom_site_general', 'tbp_lv_location', 'tbp_tile_type', 'sex']

# # 进行独热编码
# combined_df = pd.get_dummies(combined_df, columns=categorical_cols)

# # 拆分回训练集和测试集
# df_train_encoded = combined_df.iloc[:len(df_train_clean)]
# df_test_encoded = combined_df.iloc[len(df_train_clean):]

# # 还原索引
# df_train_encoded.index = df_train.index
# df_test_encoded.index = df_test.index

# # 检查处理结果
# print(df_train_encoded.head())
# print(df_test_encoded.head())
# df_train=df_train_encoded
# df_test=df_test_encoded
# df_train.to_csv("train.csv")
# df_test.to_csv("test.csv")
# import pandas as pd
# from sklearn.preprocessing import StandardScaler


# # 1.转换数据类型
# cols_to_convert = [col for col in df_train.columns if col != 'isic_id']
# print(cols_to_convert)
# df_train[cols_to_convert] = df_train[cols_to_convert].astype('float64')
# df_test[cols_to_convert] = df_test[cols_to_convert].astype('float64')

# # 2. 对 df_train 进行标准化
# scaler = StandardScaler()
# df_train_scaled = df_train.copy()
# df_test_scaled = df_test.copy()

# # 标准化训练数据
# df_train_scaled[cols_to_convert] = scaler.fit_transform(df_train_scaled[cols_to_convert])

# # 标准化测试数据
# df_test_scaled[cols_to_convert] = scaler.transform(df_test_scaled[cols_to_convert])

# # 打印数据类型和标准化后的数据摘要
# print("训练集数据类型:")
# print(df_train_scaled.dtypes)

# print("\n测试集数据类型:")
# print(df_test_scaled.dtypes)

# print("\n训练集标准化后的描述统计信息:")
# print(df_train_scaled.describe())

# print("\n测试集标准化后的描述统计信息:")
# print(df_test_scaled.describe())
# df_train_scaled.to_csv("train_scaled.csv")
# df_test_scaled.to_csv("test_scaled.csv")

## <span><h2 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 50px 50px;">Set Seed for Reproducibility</h2></span>

In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f"Seeding done ...")

def flush():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
set_seed(CONFIG['seed'])

In [ ]:
ROOT_DIR = "/kaggle/input/isic-2024-challenge"
TRAIN_DIR = f'{ROOT_DIR}/train-image/image'

In [ ]:
def get_train_file_path(image_id):
    return f"{TRAIN_DIR}/{image_id}.jpg"

# <h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Read the Data</h1>

In [ ]:
import os

os.environ['LANG'] = 'en_US.UTF-8'
os.environ['LC_ALL'] = 'en_US.UTF-8'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf resize_data
!unzip /content/drive/MyDrive/resize_data.zip


In [ ]:
train_images = sorted(glob.glob(f"{TRAIN_DIR}/*.jpg"))
print(train_images)

In [ ]:
# Read past isic competition data
check_path = lambda p: os.path.exists(p)
DATA_PATH = Path("/content/resize_data/")
# 2020
get_image_path_2020 = lambda p: os.path.join(f'{str(DATA_PATH/"Data_2020")}/{p}.jpg')
df_train_2020 = pd.read_excel(DATA_PATH/"Data_2020.xlsx")
df_positive_2020 = df_train_2020[df_train_2020["melanoma"] == 1].reset_index(drop=True)
df_positive_2020['file_path'] = df_positive_2020["image_id"].apply(get_image_path_2020)
df_positive_2020['exists'] = df_positive_2020['file_path'].apply(check_path)
df_positive_2020 = df_positive_2020[df_positive_2020['exists'] == True].reset_index()
df_negative_2020 = df_train_2020[df_train_2020["melanoma"] == 0].reset_index(drop=True)
df_negative_2020['file_path'] = df_negative_2020["image_id"].apply(get_image_path_2020)
df_negative_2020['exists'] = df_negative_2020['file_path'].apply(check_path)
df_negative_2020 = df_negative_2020[df_negative_2020['exists'] == True].reset_index()
# print(df_positive_2020)
# print(df_negative_2020)
#2019
get_image_path_2019 = lambda p: os.path.join(f'{str(DATA_PATH/"Data_2019")}/{p}.jpg')
df_train_2019 = pd.read_excel(DATA_PATH/"Data_2019.xlsx")
df_positive_2019=df_train_2019[df_train_2019["melanoma"] == 1].reset_index(drop=True)
df_positive_2019['file_path'] = df_positive_2019["image_id"].apply(get_image_path_2019)
df_positive_2019['exists'] = df_positive_2019['file_path'].apply(check_path)
df_positive_2019 = df_positive_2019[df_positive_2019['exists'] == True].reset_index()
df_negative_2019 = df_train_2019[df_train_2019["melanoma"] == 0].reset_index(drop=True)
df_negative_2019['file_path'] = df_negative_2019["image_id"].apply(get_image_path_2019)
df_negative_2019['exists'] = df_negative_2019['file_path'].apply(check_path)
df_negative_2019 = df_negative_2019[df_negative_2019['exists'] == True].reset_index()
# print(df_positive_2019)
# print(df_negative_2019)
#2018
get_image_path_2018 = lambda p: os.path.join(f'{str(DATA_PATH/"Data_2018")}/{p}.jpg')
df_train_2018 = pd.read_excel(DATA_PATH/"Data_2018.xlsx")
df_positive_2018=df_train_2018[df_train_2018["melanoma"] == 1].reset_index(drop=True)
df_positive_2018['file_path'] = df_positive_2018["image_id"].apply(get_image_path_2018)
df_positive_2018['exists'] = df_positive_2018['file_path'].apply(check_path)
df_positive_2018 = df_positive_2018[df_positive_2018['exists'] == True].reset_index()
df_negative_2018 = df_train_2018[df_train_2018["melanoma"] == 0].reset_index(drop=True)
df_negative_2018['file_path'] = df_negative_2018["image_id"].apply(get_image_path_2018)
df_negative_2018['exists'] = df_negative_2018['file_path'].apply(check_path)
df_negative_2018 = df_negative_2018[df_negative_2018['exists'] == True].reset_index()
# print(df_positive_2018)
# print(df_negative_2018)
#2017
get_image_path_2017 = lambda p: os.path.join(f'{str(DATA_PATH/"Data_2017")}/{p}.jpg')
df_train_2017 = pd.read_excel(DATA_PATH/"Data_2017.xlsx")
df_positive_2017=df_train_2017[df_train_2017["melanoma"] == 1].reset_index(drop=True)
df_positive_2017['file_path'] = df_positive_2017["image_id"].apply(get_image_path_2017)
df_positive_2017['exists'] = df_positive_2017['file_path'].apply(check_path)
df_positive_2017 = df_positive_2017[df_positive_2017['exists'] == True].reset_index()
df_negative_2017 = df_train_2017[df_train_2017["melanoma"] == 0].reset_index(drop=True)
df_negative_2017['file_path'] = df_negative_2017["image_id"].apply(get_image_path_2017)
df_negative_2017['exists'] = df_negative_2017['file_path'].apply(check_path)
df_negative_2017 = df_negative_2017[df_negative_2017['exists'] == True].reset_index()
# print(df_positive_2017)
# print(df_negative_2017)
#2016
get_image_path_2016 = lambda p: os.path.join(f'{str(DATA_PATH/"Data_2016")}/{p}.jpg')
df_train_2016 = pd.read_excel(DATA_PATH/"Data_2016.xlsx")
df_positive_2016=df_train_2016[df_train_2016["melanoma"] == 1].reset_index(drop=True)
df_positive_2016['file_path'] = df_positive_2016["image_id"].apply(get_image_path_2016)
df_positive_2016['exists'] = df_positive_2016['file_path'].apply(check_path)
df_positive_2016 = df_positive_2016[df_positive_2016['exists'] == True].reset_index()
df_negative_2016 = df_train_2016[df_train_2016["melanoma"] == 0].reset_index(drop=True)
df_negative_2016['file_path'] = df_negative_2016["image_id"].apply(get_image_path_2016)
df_negative_2016['exists'] = df_negative_2016['file_path'].apply(check_path)
df_negative_2016 = df_negative_2016[df_negative_2016['exists'] == True].reset_index()
# print(df_positive_2016)
# print(df_negative_2016)
past_df_positive=pd.concat([df_positive_2016,df_positive_2017,df_positive_2018,df_positive_2019,df_positive_2020],axis=0)
print(past_df_positive)
past_df_positive["isic_id"]=past_df_positive["image_id"]
past_df_positive["target"]=past_df_positive["melanoma"]
past_df_positive["patient_id"]=0
print(past_df_positive)
past_df_negative=pd.concat([df_negative_2016,df_negative_2017,df_negative_2018,df_negative_2019,df_negative_2020],axis=0)
past_df_negative["isic_id"]=past_df_negative["image_id"]
past_df_negative["target"]=past_df_negative["melanoma"]
past_df_negative["patient_id"]=0
print(past_df_negative)

In [ ]:
# # Read past isic competition data
# check_path = lambda p: os.path.exists(p)

# # 2020
# DATA_PATH_2020 = Path("/kaggle/input/isic-2020-jpg-256x256-resized")
# get_image_path_2020 = lambda p: os.path.join(f'{str(DATA_PATH_2020/"train-image/image")}/{p}.jpg')
# df_train_2020 = pd.read_csv(DATA_PATH_2020/"train-metadata.csv")
# df_positive_2020 = df_train_2020[df_train_2020["target"] == 1].reset_index(drop=True)
# df_positive_2020['file_path'] = df_positive_2020["isic_id"].apply(get_image_path_2020)
# df_positive_2020['exists'] = df_positive_2020['file_path'].apply(check_path)
# df_positive_2020 = df_positive_2020[df_positive_2020['exists'] == True].reset_index()
# # 2019
# DATA_PATH_2019 = Path("/kaggle/input/isic-2019-jpg-256x256-resized")
# get_image_path_2019 = lambda p: os.path.join(f'{str(DATA_PATH_2019/"train-image/image")}/{p}.jpg')
# df_train_2019 = pd.read_csv(DATA_PATH_2019/"train-metadata.csv")
# df_positive_2019 = df_train_2019[df_train_2019["target"] == 1].reset_index(drop=True)
# df_positive_2019['file_path'] = df_positive_2019["isic_id"].apply(get_image_path_2019)
# df_positive_2019['exists'] = df_positive_2019['file_path'].apply(check_path)
# df_positive_2019 = df_positive_2019[df_positive_2019['exists'] == True].reset_index()
# # 2018
# DATA_PATH_2018 = Path("/kaggle/input/isic-2018-jpg-256x256-resized")
# get_image_path_2018 = lambda p: os.path.join(f'{str(DATA_PATH_2018/"train-image/image")}/{p}.jpg')
# df_train_2018 = pd.read_csv(DATA_PATH_2018/"train-metadata.csv")
# df_positive_2018 = df_train_2018[df_train_2018["target"] == 1].reset_index(drop=True)
# df_positive_2018['file_path'] = df_positive_2018["isic_id"].apply(get_image_path_2018)
# df_positive_2018['exists'] = df_positive_2018['file_path'].apply(check_path)
# df_positive_2018 = df_positive_2018[df_positive_2018['exists'] == True].reset_index()

In [ ]:
# past_df_positive = pd.concat([df_positive_2018, df_positive_2019, df_positive_2020], axis=0)
# past_df_positive.head(3)
# print(f'Number of positive cases: {past_df_positive.shape[0]}')

In [ ]:
df = pd.read_csv(f"{ROOT_DIR}/train-metadata.csv")
print("        df.size, # of positive cases")
print("original>", df.shape[0], df.target.sum())
#df_2024 = df.copy()
df["patient_id"]=1
selected_cols = ['isic_id', 'target', 'file_path', 'patient_id']
# Positive cases
df_positive = df[df["target"] == 1].reset_index(drop=True)
df_positive['file_path'] = df_positive['isic_id'].apply(get_train_file_path)
if CONFIG['use_previous_positive_data']:
  df_positive = pd.concat([df_positive[selected_cols],past_df_positive[selected_cols]], axis = 0).reset_index(drop=True)
# Negative cases
df_negative = df[df["target"] == 0].reset_index(drop=True)
df_negative['file_path'] = df_negative['isic_id'].apply(get_train_file_path)
if CONFIG['use_previous_positive_data']:
    df_negative = pd.concat([past_df_negative[selected_cols],df_negative[selected_cols]], axis = 0).reset_index(drop=True)
    #df_negative = df_negative.iloc[:df_positive.shape[0]*55, :] # positive:negative = 1:5
else:
    df_negative = df_negative.iloc[:df_positive.shape[0]*1010, :]

df_negative = df_negative[selected_cols]
#df_negative = df_negative[ df_negative["file_path"].isin(train_images) ]
df = pd.concat([df_negative,df_positive]).reset_index(drop=True)
print("filtered>", df.shape[0], df.target.sum())

# df.head(3)

In [ ]:
df = pd.read_csv(f"{ROOT_DIR}/train-metadata.csv")
print("        df.size, # of positive cases")
print("original>", df.shape[0], df.target.sum())
df_2024 = df.copy()
df["patient_id"]=1
selected_cols = ['isic_id', 'target', 'file_path', 'patient_id']
# Positive cases
df_positive = df[df["target"] == 1].reset_index(drop=True)
df_positive['file_path'] = df_positive['isic_id'].apply(get_train_file_path)
if CONFIG['use_previous_positive_data']:
  df_positive = pd.concat([df_positive[selected_cols], past_df_positive[selected_cols]], axis = 0).reset_index(drop=True)
# Negative cases
df_negative = df[df["target"] == 0].reset_index(drop=True)
df_negative['file_path'] = df_negative['isic_id'].apply(get_train_file_path)
#if CONFIG['use_previous_positive_data']:
#    df_negative = pd.concat([past_df_negative[selected_cols],df_negative[selected_cols]], axis = 0).reset_index(drop=True)
    #df_negative = df_negative.iloc[:df_positive.shape[0]*55, :] # positive:negative = 1:5
# else:
#     df_negative = df_negative.iloc[:df_positive.shape[0]*1000, :]

df_negative = df_negative[selected_cols]
#df_negative = df_negative[ df_negative["file_path"].isin(train_images) ]
df = pd.concat([df_negative,df_positive]).reset_index(drop=True)
print("filtered>", df.shape[0], df.target.sum())
CONFIG['T_max'] = df.shape[0] * (CONFIG["n_fold"]-1) * CONFIG['epochs'] // CONFIG['train_batch_size'] // CONFIG["n_fold"]
CONFIG['T_max']

In [ ]:
CONFIG['T_max'] = df.shape[0] * (CONFIG["n_fold"]-1) * CONFIG['epochs'] // CONFIG['train_batch_size'] // CONFIG["n_fold"]
CONFIG['T_max']

## <span><h2 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 10px 10px;">Create Folds</h2></span>

In [ ]:
sgkf = StratifiedGroupKFold(n_splits=CONFIG['n_fold']) # I don't why my SGKF doesn't work without patient_id
from sklearn.model_selection import KFold
kf = KFold(CONFIG['n_fold'], shuffle=True)
print(len(df))
for fold, ( _, val_) in enumerate(kf.split(df)):
    df.loc[val_ , "kfold"] = int(fold)
    print("Validation indices for the first fold:", val_)
print(df["kfold"])


## <span><h2 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 10px 10px;">Dataset Class</h2></span>

In [ ]:
from albumentations import RandomBrightnessContrast, Compose, RGBShift
import cv2
import random
class ISICDataset_for_Train(Dataset):
    def __init__(self, df, transforms=None):
        print("current or pass",len(df),len(df[df["patient_id"] == 1]),len(df[df["patient_id"] == 0]))
        self.df_positive = df[df["target"] == 1].reset_index()
        self.df_negative = df[df["target"] == 0].reset_index()
        self.file_names_positive = self.df_positive['file_path'].values
        self.file_names_negative = self.df_negative['file_path'].values
        self.targets_positive = self.df_positive['target'].values
        self.targets_negative = self.df_negative['target'].values
        self.past_df_positive = self.df_positive["patient_id"].values
        self.past_df_negative = self.df_negative["patient_id"].values
        self.transforms = transforms
        self.total_len = len(df)

        df_=df[df["patient_id"] == 1].reset_index()
        print("postive or negative",len(df_),len(df_[df_["target"] == 1]),len(df_[df_["target"] == 0]))
        print(self.total_len)
        self.transform = A.Compose([
            RandomBrightnessContrast (brightness_limit=(-0.1, -0.1), brightness_by_max=True, always_apply=None, p=1),
            RGBShift(r_shift_limit=(35, 35), g_shift_limit=(30, 30), b_shift_limit=(-10, -10), p=1.0)
        ])

    def __len__(self):
        return self.total_len #len(self.df_positive) * 4

    def __getitem__(self, index):
        if random.random() <= 0.2:
          df = self.df_positive
          file_names = self.file_names_positive
          targets = self.targets_positive
          past=self.past_df_positive


        else:
            df = self.df_negative
            file_names = self.file_names_negative
            targets = self.targets_negative
            past=self.past_df_negative

        index = index % df.shape[0]
        is_past=past[index]
        img_path = file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = targets[index]

        if self.transforms:
          if target==1:
            #if is_past==0:
            #  img = self.transform(image=img)["image"]

            img = self.transforms['with_augm'](image=img)["image"]
          else:
            img = self.transforms['no_augm'](image=img)["image"]


        return {
            'image': img,
            'target': target
        }
class ISIC_2024_Dataset_for_train(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.targets = df['target'].values
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = self.targets[index]

        if self.transforms:
            img = self.transforms['no_augm'](image=img)["image"]

        return {
            'image': img,
            'target': target
        }

# class ISIC_2024_Dataset_for_test(Dataset):
#     def __init__(self, df, transforms=None):
#         self.df = df
#         self.file_names = df['file_path'].values
#         self.targets = df['target'].values
#         self.transforms = transforms

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, index):
#         img_path = self.file_names[index]
#         img = cv2.imread(img_path)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         target = self.targets[index]

#         if self.transforms:
#             img = self.transforms['no_augm'](image=img)["image"]

#         return {
#             'image': img,
#             'target': target
#         }
class ISICDataset(Dataset):
    def __init__(self, df, transforms=None):
        print("total current or pass",len(df),len(df[df["patient_id"] == 1]),len(df[df["patient_id"] == 0]))
        self.df = df[df["patient_id"] == 1].reset_index()
        # self.df=df
        print("postive or negative",len(self.df),len(self.df[self.df["target"] == 1]),len(self.df[self.df["target"] == 0]))
        self.file_names = df['file_path'].values
        self.targets = df['target'].values
        self.transforms = transforms
        print("len",len(df),len(self.df))

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = self.targets[index]
        if self.transforms:
          img = self.transforms['no_augm'](image=img)["image"]

        # if self.transforms:
        #     img = self.transforms(image=img)["image"]

        return {
            'image': img,
            'target': target
        }

## <span><h2 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 20px 20px">Augmentations</h2></span>

In [ ]:

# data_transforms = {
#     "with_augm": A.Compose([
#         A.Resize(CONFIG['img_size'], CONFIG['img_size']),
#         A.RandomRotate90(p=0.5),
#         A.Flip(p=0.5),
#         A.Downscale(p=0.25),
#         A.ShiftScaleRotate(shift_limit=0.1,
#                            scale_limit=0.15,
#                            rotate_limit=60,
#                            p=0.5),
#         A.HueSaturationValue(
#                 hue_shift_limit=0.1,
#                 sat_shift_limit=0.1,
#                 val_shift_limit=0.1,
#                 p=0.5
#             ),
#         A.RandomBrightnessContrast(
#                 brightness_limit=(-0.1,0.1),
#                 contrast_limit=(-0.1, 0.1),
#                 p=0.5
#             ),
#         A.Normalize(
#                 mean=[0.485, 0.456, 0.406],
#                 std=[0.229, 0.224, 0.225],
#                 max_pixel_value=255.0,
#                 p=1.0
#             ),
#         ToTensorV2()], p=1.),

#     "no_augm": A.Compose([
#         A.Resize(CONFIG['img_size'], CONFIG['img_size']),
#         A.Normalize(
#                 mean=[0.485, 0.456, 0.406],
#                 std=[0.229, 0.224, 0.225],
#                 max_pixel_value=255.0,
#                 p=1.0
#             ),
#         ToTensorV2()], p=1.)
# }

In [ ]:

data_transforms = {
    "with_augm": A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Transpose(p=0.2),
    A.VerticalFlip(p=0.25),
    A.HorizontalFlip(p=0.25),
    A.RandomRotate90(p=0.2),
    # A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1, p=0.25),
    A.Downscale(p=0.2),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.25),
    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=0.25),
    # A.CLAHE(clip_limit=4.0, p=0.2),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.25),
    A.RandomBrightnessContrast(
        brightness_limit=(-0.2, 0.2),
        contrast_limit=(-0.2, 0.2),
        p=0.2
    ),
    A.ShiftScaleRotate(
        shift_limit=0.1,
        scale_limit=0.15,
        rotate_limit=60,
        border_mode=0,
        p=0.2
    ),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0
    ),
    ToTensorV2(),
], p=1.0),
    "simple_augm":A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Transpose(p=0.2),
    A.VerticalFlip(p=0.2),
    A.HorizontalFlip(p=0.2),
    A.RandomRotate90(p=0.2),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0
    ),
    ToTensorV2(),
], p=1.0),
    "no_augm": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Create Model</h1></span>
## <span><h2 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px">GeM Pooling</h2></span>

In [ ]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)

    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'
# class ISICModel(nn.Module):
#     def __init__(self, model_name, num_classes=1, pretrained=True, checkpoint_path=None):
#         super(ISICModel, self).__init__()
#         self.Relu=nn.ReLU()
#         self.dropout=nn.Dropout(0.1)
#         self.model = timm.create_model("swin_base_patch4_window7_224", pretrained=pretrained, checkpoint_path=checkpoint_path)
#         self.model2 = timm.create_model("convnext_base", pretrained=pretrained, checkpoint_path=checkpoint_path)
#         #print(self.model)
#         #print(self.model2)
#         in_features = self.model2.head.fc.in_features+self.model.head.fc.in_features
#         self.model.head.fc = nn.Identity()
#         self.model2.head.fc=nn.Identity()

#         # self.pooling = GeM()
#         # self.linear = nn.Linear(in_features, num_classes)
#         self.sigmoid = nn.Sigmoid()
#         self.nn=nn.Linear(in_features,1024)
#         self.nn2 = nn.Linear(1024, num_classes)

#     def forward(self, images):
#         output = self.model(images)
#         output1=self.model2(images)

#         output=torch.cat((output1, output), dim=1)
#         output=self.Relu(output)
#         output=self.dropout(output)
#         output=self.nn(output)
#         output=self.Relu(output)
#         output=self.dropout(output)
#         output=self.nn2(output)


#         # output = self.linear(features)
#         # output = self.sigmoid(output)
#         # print(output.shape)
#         return output
# class ISICModel(nn.Module):
#     def __init__(self, model_name, num_classes=1, pretrained=True, checkpoint_path=None):
#         super(ISICModel, self).__init__()
#         self.Relu=nn.ReLU()
#         self.dropout=nn.Dropout(0.1)
#         self.model = timm.create_model("swin_base_patch4_window7_224", pretrained=pretrained, checkpoint_path=checkpoint_path)
#         self.model2 = timm.create_model("convnext_base", pretrained=pretrained, checkpoint_path=checkpoint_path)
#         # print(self.model)
#         # print(self.model2)
#         in_features = self.model2.classifier.in_features+self.model.head.fc.in_features
#         self.model.head.fc = nn.Identity()
#         self.model2.classifier = nn.Identity()

#         # self.pooling = GeM()
#         # self.linear = nn.Linear(in_features, num_classes)
#         self.sigmoid = nn.Sigmoid()
#         self.nn=nn.Linear(in_features,1024)
#         self.nn2 = nn.Linear(1024, num_classes)

#     def forward(self, images):
#         output = self.model(images)
#         output1=self.model2(images)

#         output=torch.cat((output1, output), dim=1)
#         output=self.Relu(output)
#         output=self.dropout(output)
#         output=self.nn(output)
#         output=self.Relu(output)
#         output=self.dropout(output)
#         output=self.nn2(output)


#         # output = self.linear(features)
#         # output = self.sigmoid(output)
#         # print(output.shape)
#         return output
class ISICModel(nn.Module):
    def __init__(self, model_name, num_classes=1, pretrained=True, checkpoint_path=None):
        super(ISICModel, self).__init__()
        self.Relu=nn.ReLU()
        self.dropout=nn.Dropout(0.1)
        self.model = timm.create_model("swin_base_patch4_window7_224", pretrained=pretrained, checkpoint_path=checkpoint_path)
        self.model2 = timm.create_model("efficientnet_b0", pretrained=pretrained, checkpoint_path=checkpoint_path)
        # print(self.model)
        # print(self.model2)
        in_features = self.model2.classifier.in_features+self.model.head.fc.in_features
        self.model.head.fc = nn.Identity()
        self.model2.classifier = nn.Identity()

        # self.pooling = GeM()
        # self.linear = nn.Linear(in_features, num_classes)
        self.sigmoid = nn.Sigmoid()
        self.nn=nn.Linear(in_features,1024)
        self.nn2 = nn.Linear(1024, num_classes)

    def forward(self, images):
        output = self.model(images)
        output1=self.model2(images)

        output=torch.cat((output1, output), dim=1)
        output=self.Relu(output)
        output=self.dropout(output)
        output=self.nn(output)
        output=self.Relu(output)
        output=self.dropout(output)
        output=self.nn2(output)


        # output = self.linear(features)
        # output = self.sigmoid(output)
        # print(output.shape)
        return output
torch.cuda.empty_cache()
model = ISICModel(CONFIG['model_name'])
model.to(CONFIG['device']);
#print(model)

# <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px;">Loss Function</h1></span>

In [ ]:

def criterion(input, target, alpha=0.9, gamma=5):
  bce_loss = F.binary_cross_entropy_with_logits(input, target, reduction='none')
  pt = torch.exp(-bce_loss)
  loss = alpha * (1 - pt) ** gamma * bce_loss
  return loss.mean()

# <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 20px 20px">Training Function</h1></span>

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()

    dataset_size = 0
    running_loss = 0.0
    running_auroc  = 0.0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.float)

        batch_size = images.size(0)

        outputs = model(images).squeeze()
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']

        loss.backward()

        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()

        auroc = binary_auroc(input=outputs.squeeze(), target=targets).item()

        running_loss += (loss.item() * batch_size)
        running_auroc  += (auroc * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size

        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, Train_Auroc=epoch_auroc,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()

    return epoch_loss, epoch_auroc

# <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 20px 20px">Validation Function</h1></span>

In [ ]:
@torch.inference_mode()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0.0
    running_auroc = 0.0
    y_true = []
    y_pred = []

    auroc_metric = AUROC(task="binary").to(device)

    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.float)

        batch_size = images.size(0)
        outputs = model(images).squeeze()
        loss = criterion(outputs, targets)
        auroc = auroc_metric(torch.sigmoid(outputs), targets.int()).item()

        y_true.extend(targets.cpu().numpy())
        y_pred.extend(torch.sigmoid(outputs).cpu().detach().numpy())
        #y_pred.extend(outputs.cpu().detach().numpy())
        running_loss += (loss.item() * batch_size)
        running_auroc += (auroc * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size

        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss, Valid_Auroc=epoch_auroc,
                        LR=optimizer.param_groups[0]['lr'])

    cm = confusion_matrix(y_true, [1 if p > 0.5 else 0 for p in y_pred])

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - Epoch {epoch}')
    plt.show()

    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic - Epoch {epoch}')
    plt.legend(loc="lower right")
    plt.show()

    gc.collect()

    return epoch_loss, epoch_auroc

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Training</h1></span>

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_auroc = -np.inf
    best_valid_loss = np.inf
    history = defaultdict(list)

    for epoch in range(1, num_epochs + 1):
        gc.collect()
        train_epoch_loss, train_epoch_auroc = train_one_epoch(model, optimizer, scheduler,
                                           dataloader=train_loader,
                                           device=CONFIG['device'], epoch=epoch)

        val_epoch_loss, val_epoch_auroc = valid_one_epoch(model, valid_loader, device=CONFIG['device'],
                                         epoch=epoch)

        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Train AUROC'].append(train_epoch_auroc)
        history['Valid AUROC'].append(val_epoch_auroc)
        history['lr'].append( scheduler.get_lr()[0] )
        if val_epoch_loss <= best_valid_loss:
            print(f"Validation Loss Improved ({best_valid_loss} ---> {val_epoch_loss})")
            best_valid_loss = val_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "best_loss_conv_swin_withargu_oversample.pt"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved")

        if best_epoch_auroc <= val_epoch_auroc:
            print(f"Validation AUROC Improved ({best_epoch_auroc} ---> {val_epoch_auroc})")
            best_epoch_auroc = val_epoch_auroc
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "best_auroc_conv_swin_withargu_oversample.pt"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved")



        print()

    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best AUROC: {:.4f}".format(best_epoch_auroc))
    print("Best Loss: {:.4f}".format(best_valid_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, history

In [ ]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'],
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'],
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None

    return scheduler

# <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 20px 20px">Prepare Dataloader</h1></span>

In [ ]:
###实际上用两个都是ISICDataset效果可能更好
def prepare_loaders(df, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    train_dataset = ISICDataset_for_Train(df_train, transforms=data_transforms)
    ##train_dataset = ISICDataset(df_train, transforms=data_transforms)
    valid_dataset = ISICDataset(df_valid, transforms=data_transforms)
    #train_dataset = ISIC_2024_Dataset_for_train(df_train, transforms=data_transforms)
    #valid_dataset = ISICDataset(df_valid, transforms=data_transforms)
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'],
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'],
                              num_workers=2, shuffle=False, pin_memory=True)

    return train_loader, valid_loader

In [ ]:
train_loader, valid_loader = prepare_loaders(df, fold=0)


## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 20px 20px">Define Optimizer and Scheduler</h1></span>

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'],
                       weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

## <span><h1 style = "font-family: garamond; font-size: 30px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 20px 20px">Start Training</h1></span>

In [ ]:
model, history = run_training(model, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=CONFIG['epochs'])

In [ ]:
from google.colab import files
torch.save(model.state_dict(),"swin224.ckpt")
files.download('swin224.ckpt')

In [ ]:
history = pd.DataFrame.from_dict(history)
history.to_csv("history.csv", index=False)

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Logs</h1></span>

In [ ]:
plt.plot( range(history.shape[0]), history["Train Loss"].values, label="Train Loss")
plt.plot( range(history.shape[0]), history["Valid Loss"].values, label="Valid Loss")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.plot( range(history.shape[0]), history["Train AUROC"].values, label="Train AUROC")
plt.plot( range(history.shape[0]), history["Valid AUROC"].values, label="Valid AUROC")
plt.xlabel("epochs")
plt.ylabel("AUROC")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.plot( range(history.shape[0]), history["lr"].values, label="lr")
plt.xlabel("epochs")
plt.ylabel("lr")
plt.grid()
plt.legend()
plt.show()